# Experiments with NWPU satellite images

In [1]:
import os
import numpy as np
import pandas as pd
import json
import cv2
import random

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


## Load images

In [2]:
def load_images(base_path, classes, num_samples, train=True):
    images = []
    labels = []

    subset = 'train/train' if train else 'test/test'
    base_path = os.path.join(base_path, subset)

    for cls in classes:
        cls_path = os.path.join(base_path, cls)
        img_files = os.listdir(cls_path)
        
        # Pick random images from the class
        selected_files = random.sample(img_files, num_samples)
        
        for img_file in selected_files:
            img_path = os.path.join(cls_path, img_file)
            img = cv2.imread(img_path)
            img = cv2.resize(img, (224, 224))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            images.append(img)
            labels.append(cls)

    return images, labels

base_path = 'dataset/NWPU-RESISC45'
classes = ['forest', 'river', 'meadow', 'dense_residential', 'industrial_area', 'freeway', 'mountain', 'island', 'circular_farmland', 'rectangular_farmland']
num_samples = 500

images, labels = load_images(base_path, classes, num_samples)

In [3]:
mapping = {
    'forest': 'Forest',
    'river': 'River',
    'dense_residential': 'Residential',
    'meadow': 'Pasture',
    'industrial_area': 'Industrial',
    'freeway': 'Highway',
    'mountain': 'HerbaceousVegitation', 
    'island': 'SeaLake', 
    'circular_farmland': 'PermanentCrop', 
    'rectangular_farmland': 'AnnualCrop',
}
mapped_labels = [mapping[label] for label in labels]


In [4]:
# Preprocess the images and labels
images = np.array(images) / 255.0
labels_encoded = pd.get_dummies(mapped_labels).values
y_true = np.argmax(labels_encoded, axis=1)

## ConvNeXt

In [5]:
## load model
from tensorflow.keras.models import load_model
model_1 = load_model("model_1_nFull_epoch5_batch32")

In [6]:
# Evaluate the model in batches
batch_size = 32
y_pred_batches = []

for i in range(0, len(images), batch_size):
    batch_pred = model_1.predict(images[i:i + batch_size])
    y_pred_batches.append(batch_pred)

y_pred = np.concatenate(y_pred_batches, axis=0)
y_pred_classes = np.argmax(y_pred, axis=1)


1/1 [==============================] - 2s 2s/step


In [7]:
# Evaluate the performance
accuracy = accuracy_score(y_true, y_pred_classes)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 54.38%


In [8]:
class_names = list(pd.get_dummies(mapped_labels).columns)
model_class_indices = list(range(10))  # List of indices for the 10 classes in the model

In [9]:
print("\nClassification Report:")
print(classification_report(y_true, y_pred_classes, target_names=class_names, labels=model_class_indices))


Classification Report:
                      precision    recall  f1-score   support

          AnnualCrop       0.37      0.25      0.30       500
              Forest       0.60      0.77      0.68       500
HerbaceousVegitation       0.76      0.87      0.81       500
             Highway       0.63      0.67      0.65       500
          Industrial       0.55      0.77      0.64       500
             Pasture       0.20      0.10      0.14       500
       PermanentCrop       0.34      0.43      0.38       500
         Residential       0.59      0.17      0.26       500
               River       0.66      0.51      0.58       500
             SeaLake       0.56      0.89      0.69       500

            accuracy                           0.54      5000
           macro avg       0.53      0.54      0.51      5000
        weighted avg       0.53      0.54      0.51      5000



In [10]:
print("\nConfusion Matrix:")
print(confusion_matrix(y_true, y_pred_classes))


Confusion Matrix:
[[125   1   3  24   4 100 214  13   8   8]
 [  0 387  43  16   2   3   1   1  11  36]
 [  0  48 437   1   0   7   0   0   0   7]
 [  2   1   1 333  42   3  21   0  22  75]
 [  0   0   0  41 383   0  29  36  10   1]
 [  3 183   8  42   1  52   0   2  15 194]
 [209   0   1  11   0  57 217   0   0   5]
 [  0   0   0  13 261   4  95  85  40   2]
 [  0  21  65  49   1  32  53   6 257  16]
 [  0   0  20   0   0   6   3   0  28 443]]


In [11]:
cm = confusion_matrix(y_true, y_pred_classes)
cm_df = pd.DataFrame(cm, index=class_names, columns=class_names)

In [12]:
# ## My matplotlip is broken so im using something else. 
# ## Feel free to use this if you want. 

# import seaborn as sns
# import matplotlib.pyplot as plt

# plt.figure(figsize=(10, 7))
# sns.set(font_scale=1.4)  # Adjust font size
# sns.heatmap(cm_df, annot=True, annot_kws={"size": 12}, cmap="YlGnBu", fmt="g", linewidths=.5, cbar=False)
# plt.ylabel('True label')
# plt.xlabel('Predicted label')
# plt.title('Confusion Matrix Heatmap')

# plt.show()


In [38]:
## using this because my matplotlib is broken
## feel free to use the block from above if yours work. 

import plotly.figure_factory as ff

z = cm.tolist()  # Convert the confusion matrix to a nested list
z_text = [["{:.0f}".format(y) for y in x] for x in z]  # Format values as strings

fig = ff.create_annotated_heatmap(
    z=z,
    x=class_names,
    y=class_names,
    annotation_text=z_text,
    colorscale="Viridis",
    showscale=True
)

fig.update_layout(
    title="Confusion Matrix Heatmap",
    xaxis_title="Predicted label",
    yaxis_title="True label",
    font=dict(size=12)
)

fig.show()


## ViT

In [14]:
def load_images(base_path, classes, num_samples, train=True):
    images = []
    labels = []

    subset = 'train/train' if train else 'test/test'
    base_path = os.path.join(base_path, subset)

    for cls in classes:
        cls_path = os.path.join(base_path, cls)
        img_files = os.listdir(cls_path)
        
        # Pick random images from the class
        selected_files = random.sample(img_files, num_samples)
        
        for img_file in selected_files:
            img_path = os.path.join(cls_path, img_file)
            img = cv2.imread(img_path)
            img = cv2.resize(img, (64, 64))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            images.append(img)
            labels.append(cls)

    return images, labels

base_path = 'dataset/NWPU-RESISC45'
classes = ['forest', 'river', 'meadow', 'dense_residential', 'industrial_area', 'freeway', 'mountain', 'island', 'circular_farmland', 'rectangular_farmland']
num_samples = 500

images_vit, labels = load_images(base_path, classes, num_samples)

In [15]:
# Preprocess the images and labels
images_vit = np.array(images_vit) / 255.0
labels_encoded = pd.get_dummies(mapped_labels).values
y_true = np.argmax(labels_encoded, axis=1)

In [16]:
import tensorflow_addons as tfa
from tensorflow.keras.models import load_model

custom_objects = {
    "AdamW": tfa.optimizers.AdamW
}

model_2 = load_model("model_vit_classifier", custom_objects=custom_objects)

c:\Users\Geran\anaconda3\envs\tfgpu_landclass\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning:



TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 




In [17]:
# Evaluate the model in batches
batch_size = 32
y_pred_batches = []

for i in range(0, len(images_vit), batch_size):
    batch_pred = model_2.predict(images_vit[i:i + batch_size])
    y_pred_batches.append(batch_pred)

y_pred = np.concatenate(y_pred_batches, axis=0)
y_pred_classes = np.argmax(y_pred, axis=1)


1/1 [==============================] - 1s 728ms/step


In [18]:
# Evaluate the performance
accuracy = accuracy_score(y_true, y_pred_classes)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 26.70%


In [19]:
print("\nClassification Report:")
print(classification_report(y_true, y_pred_classes, target_names=class_names, labels=model_class_indices))


Classification Report:
                      precision    recall  f1-score   support

          AnnualCrop       0.16      0.03      0.05       500
              Forest       0.34      0.61      0.43       500
HerbaceousVegitation       0.13      0.50      0.21       500
             Highway       0.14      0.09      0.11       500
          Industrial       0.54      0.46      0.49       500
             Pasture       0.03      0.00      0.01       500
       PermanentCrop       0.24      0.42      0.30       500
         Residential       0.72      0.19      0.30       500
               River       0.34      0.06      0.10       500
             SeaLake       0.91      0.32      0.47       500

            accuracy                           0.27      5000
           macro avg       0.35      0.27      0.25      5000
        weighted avg       0.35      0.27      0.25      5000



In [20]:
print("\nConfusion Matrix:")
print(confusion_matrix(y_true, y_pred_classes))


Confusion Matrix:
[[ 16  54 238   3   2  12 169   4   0   2]
 [  9 303 160   2   0   0  26   0   0   0]
 [  7  40 250   2   1   2 197   1   0   0]
 [  2   2 416  43  12   0  22   2   1   0]
 [  0   0 113 100 229   0  29  27   2   0]
 [ 14 432  47   0   0   2   5   0   0   0]
 [ 22  13 208   7   1  16 208   0  17   8]
 [  0   0 222  71  37   0  76  94   0   0]
 [ 16  44 239  12   3  19 128   2  31   6]
 [ 16   8  40  67 143  10  16   1  40 159]]


In [21]:
cm = confusion_matrix(y_true, y_pred_classes)
cm_df = pd.DataFrame(cm, index=class_names, columns=class_names)

In [22]:
## using this because my matplotlib is broken
## feel free to use the block from above if yours work. 

import plotly.figure_factory as ff

z = cm.tolist()  # Convert the confusion matrix to a nested list
z_text = [["{:.0f}".format(y) for y in x] for x in z]  # Format values as strings

fig = ff.create_annotated_heatmap(
    z=z,
    x=class_names,
    y=class_names,
    annotation_text=z_text,
    colorscale="Viridis",
    showscale=True
)

fig.update_layout(
    title="Confusion Matrix Heatmap",
    xaxis_title="Predicted label",
    yaxis_title="True label",
    font=dict(size=12)
)

fig.show()


## ResNet50

In [23]:
## load model
from tensorflow.keras.models import load_model
model_resnet = load_model("model_resnet50_nFull_epoch20_batch32_lr.00001")

In [24]:
# Evaluate the model in batches
batch_size = 32
y_pred_batches = []

for i in range(0, len(images), batch_size):
    batch_pred = model_resnet.predict(images[i:i + batch_size])
    y_pred_batches.append(batch_pred)

y_pred = np.concatenate(y_pred_batches, axis=0)
y_pred_classes = np.argmax(y_pred, axis=1)
 

1/1 [==============================] - 1s 697ms/step


In [25]:
# Evaluate the performance
accuracy = accuracy_score(y_true, y_pred_classes)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 12.30%


In [26]:
class_names = list(pd.get_dummies(classes).columns)

In [27]:
cm = confusion_matrix(y_true, y_pred_classes)
cm_df = pd.DataFrame(cm, index=class_names, columns=class_names)

In [28]:
print("\nClassification Report:")
print(classification_report(y_true, y_pred_classes, target_names=class_names))


Classification Report:
                      precision    recall  f1-score   support

   circular_farmland       0.10      0.10      0.10       500
   dense_residential       0.00      0.00      0.00       500
              forest       0.00      0.00      0.00       500
             freeway       0.10      0.09      0.09       500
     industrial_area       0.24      0.48      0.32       500
              island       0.72      0.04      0.08       500
              meadow       0.10      0.00      0.01       500
            mountain       0.08      0.39      0.13       500
rectangular_farmland       0.00      0.00      0.00       500
               river       0.16      0.14      0.15       500

            accuracy                           0.12      5000
           macro avg       0.15      0.12      0.09      5000
        weighted avg       0.15      0.12      0.09      5000



In [29]:
print("\nConfusion Matrix:")
print(confusion_matrix(y_true, y_pred_classes))


Confusion Matrix:
[[ 48   4   0  49  39   2   2 243   2 111]
 [ 73   0   2  18  42   2   3 348   0  12]
 [ 35   0   0  31  71   0   7 334   0  22]
 [ 31   8   0  44  49   0   3 249   1 115]
 [  1   2   0   7 238   0   0 225   0  27]
 [196   7   0  21   0  21   1 245   0   9]
 [ 43   5   0  82 152   0   2 186   5  25]
 [  0  33   0   0 246   0   0 194   0  27]
 [ 19   1   0  76  90   1   2 298   0  13]
 [ 50  15   0 113  78   3   0 173   0  68]]


In [30]:
## using this because my matplotlib is broken
## feel free to use the block from above if yours work. 

import plotly.figure_factory as ff

z = cm.tolist()  # Convert the confusion matrix to a nested list
z_text = [["{:.0f}".format(y) for y in x] for x in z]  # Format values as strings

fig = ff.create_annotated_heatmap(
    z=z,
    x=class_names,
    y=class_names,
    annotation_text=z_text,
    colorscale="Viridis",
    showscale=True
)

fig.update_layout(
    title="Confusion Matrix Heatmap",
    xaxis_title="Predicted label",
    yaxis_title="True label",
    font=dict(size=12)
)

fig.show()


## MobileNet

In [31]:
## load model
from tensorflow.keras.models import load_model
model_mobilenet = load_model("model_MobileNet_nFull_epoch10_batch32")

In [32]:
# Evaluate the model in batches
batch_size = 32
y_pred_batches = []

for i in range(0, len(images), batch_size):
    batch_pred = model_mobilenet.predict(images[i:i + batch_size])
    y_pred_batches.append(batch_pred)

y_pred = np.concatenate(y_pred_batches, axis=0)
y_pred_classes = np.argmax(y_pred, axis=1)


1/1 [==============================] - 0s 305ms/step


In [33]:
# Evaluate the performance
accuracy = accuracy_score(y_true, y_pred_classes)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 38.12%


In [34]:
cm = confusion_matrix(y_true, y_pred_classes)
cm_df = pd.DataFrame(cm, index=class_names, columns=class_names)

In [35]:
class_names = list(pd.get_dummies(classes).columns)
print("\nClassification Report:")
print(classification_report(y_true, y_pred_classes, target_names=class_names))


Classification Report:
                      precision    recall  f1-score   support

   circular_farmland       0.26      0.03      0.05       500
   dense_residential       0.19      0.19      0.19       500
              forest       0.54      0.95      0.69       500
             freeway       0.25      0.39      0.30       500
     industrial_area       0.50      0.88      0.63       500
              island       0.17      0.00      0.01       500
              meadow       0.29      0.46      0.36       500
            mountain       0.26      0.34      0.30       500
rectangular_farmland       0.32      0.15      0.20       500
               river       0.95      0.42      0.58       500

            accuracy                           0.38      5000
           macro avg       0.37      0.38      0.33      5000
        weighted avg       0.37      0.38      0.33      5000



In [36]:
print("\nConfusion Matrix:")
print(confusion_matrix(y_true, y_pred_classes))


Confusion Matrix:
[[ 13   3   0  24  14   4 434   8   0   0]
 [  0  97 182   1   7   0  11 199   1   2]
 [  0   6 477   8   0   0   7   2   0   0]
 [ 18  11   9 193  71   0  31  80  79   8]
 [  0   0   0   3 440   0   2  55   0   0]
 [ 10 340  24  53   8   2  37  22   4   0]
 [  9   1   5 218  23   1 230  13   0   0]
 [  0   0   1  12 307   0   8 172   0   0]
 [  0   8  92 220  12   1  27  65  74   1]
 [  0  50  90  39   5   4   0  34  70 208]]


In [37]:
## using this because my matplotlib is broken
## feel free to use the block from above if yours work. 

import plotly.figure_factory as ff

z = cm.tolist()  # Convert the confusion matrix to a nested list
z_text = [["{:.0f}".format(y) for y in x] for x in z]  # Format values as strings

fig = ff.create_annotated_heatmap(
    z=z,
    x=class_names,
    y=class_names,
    annotation_text=z_text,
    colorscale="Viridis",
    showscale=True
)

fig.update_layout(
    title="Confusion Matrix Heatmap",
    xaxis_title="Predicted label",
    yaxis_title="True label",
    font=dict(size=12)
)

fig.show()
